In [ ]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
import base64

import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
import re

# Imports CRUD class from .py module
from MongoDB_Module import AnimalShelter



###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "192234"

# Creates new CRUD object
shelter = AnimalShelter(username, password)

# Creates new DataFrame object
df = pd.DataFrame.from_records(shelter.read({}))

                               

#########################
# Dashboard Layout / View
#########################
# Creates new Dash object (screen)
app = dash.Dash('Animal Rescue Dashboard - Justin Holmes')

image_filename = 'Grazioso_Logo.png'
encoded_image = base64.b64encode(open(image_filename,'rb').read())

# Sets up Dash object layout (screen layout)
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
             style={'width': '100px', 'height': '100px'},),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    
    # HTML details for rescue specialty dropdown
    html.Select([
                 html.Option('Please choose a rescue category', id='rescue-option'),
                 html.Option('Water Rescue', id='rescue-water', n_clicks_timestamp = 0),
                 html.Option('Mountain/Wilderness Rescue', id='rescue-wilderness', n_clicks_timestamp = 0),
                 html.Option('Disaster Rescue/Individual Tracking', id='rescue-disaster', n_clicks_timestamp = 0),
                 html.Option('Reset Filters', id='rescue-reset', n_clicks_timestamp = 0),
                ],
                 id='select'),
    html.Hr(),
                  
    # HTML details for DataTable
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True}
            for i in df.columns
                ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        row_deletable=False,
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    
    html.Center(html.B(html.H1('AAC Animal Breed Percentage and Geolocation'))),
    html.Hr(),
    # HTML details for Pie Chart and Map section
    html.Div(className='row',
             style={'display':'flex'},
             children=[
                 html.Div(id='graph',
                          className='col s12 m6'),
                 html.Div(id='map-id',
                          className='col s12 m6')
             ]),
    
    # Unique Identifier
    html.Hr(),
    html.Center(html.B(html.H3('Animal Locator, Creator: Justin Holmes')))
])



#############################################
# Interaction Between Components / Controller
#############################################

# Sorts display screen by best rescue animal criteria
@app.callback(
    Output('datatable-id', "data"),
    [
        Input('rescue-water', 'n_clicks_timestamp'),
        Input('rescue-wilderness', 'n_clicks_timestamp'),
        Input('rescue-disaster', 'n_clicks_timestamp'),
        Input('rescue-reset', 'n_clicks_timestamp'),
    ]
)
def on_click(rescue_water, rescue_wilderness, rescue_disaster, rescue_reset):
    
    df = pd.DataFrame.from_records(shelter.read({}))
    
    # Returns Water Rescue animal criteria if Water option chosen
    if (int(rescue_water) > int(rescue_wilderness) and
        int(rescue_water) > int(rescue_disaster) and
        int(rescue_water) > int(rescue_reset)):
            #Criteria for Water Rescue animal
            df = pd.DataFrame.from_records(shelter.read({'breed':{'$in':[re.compile('Labrador Retriever'),
                                                                         re.compile('Chesa'),
                                                                         re.compile('Newfoundland')]},
                                                         'sex_upon_outcome':'Intact Female',
                                                         'age_upon_outcome_in_weeks':{'$gt':26, '$lt':156}}))
            
    # Returns Wilderness Rescue animal criteria if Wilderness option chosen
    elif (int(rescue_wilderness) > int(rescue_water) and 
          int(rescue_wilderness) > int(rescue_disaster) and
          int(rescue_wilderness) > int(rescue_reset)):
            # Criteria for Wilderness Rescue animal
            df = pd.DataFrame.from_records(shelter.read({'breed':{'$in':[re.compile('German Shepherd'),
                                                                         re.compile('Alaskan Malamute'),
                                                                         re.compile('Old English'),
                                                                         re.compile('Siberian Husky'),
                                                                         re.compile('Rottweiler')]},
                                                         'sex_upon_outcome':'Intact Male',
                                                         'age_upon_outcome_in_weeks':{'$gt':26, '$lt':156}}))
    # Returns Disaster Rescue animal criteria if Disaster option chosen
    elif (int(rescue_disaster) > int(rescue_water) and
          int(rescue_disaster) > int(rescue_wilderness) and
          int(rescue_disaster) > int(rescue_reset)):
            # Criteria for Disaster Rescue animal
            df = pd.DataFrame.from_records(shelter.read({'breed':{'$in':[re.compile('Dober'),
                                                                         re.compile('Blood'),
                                                                         re.compile('Germa'),
                                                                         re.compile('Golden')]},
                                                         'sex_upon_outcome':'Intact Male',
                                                         'age_upon_outcome_in_weeks':{'$gt':20, '$lt':300}}))
            print(df)
    # Returns entire database if Reset option chosen
    elif (int(rescue_reset) > int(rescue_water) and
          int(rescue_reset) > int(rescue_wilderness) and 
          int(rescue_reset) > int(rescue_disaster)):
        
            df = pd.DataFrame.from_records(shelter.read({}))

    return df.to_dict('records')


# Returns a pie graph with the of the percentage of breeds 
#sorted dashboard
@app.callback(
    Output('graph', "children"),
    [Input('datatable-id', "data"),]
)
def update_chart(viewData):
    dfs = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
            figure = px.pie(dfs, names="breed", title='Percentage of breed')
        )]


# When a row is selected the map is updated with
# new map marker, breed details, and name
@app.callback(
    Output('map-id', "children"),
    [
        Input('datatable-id', "derived_virtual_data"),
        Input('datatable-id', "selected_rows")
    ]
)
def update_map(viewData, selected_row_index):
    # Creates new DataFrame object
    dff = pd.DataFrame.from_dict(viewData)
    
    # Collects the selected row's latitude and longitude coords and assigns
    # them to local variables
    if (len(selected_row_index) is 0):
        selectionLat = float(30.75)
        selectionLon = float(-97.48)
        popupTag = 'Austin Animal Center'
    else:
        selectionLat = float(dff.loc[selected_row_index, 'location_lat'])
        selectionLon = float(dff.loc[selected_row_index, 'location_long'])
        popupTag = 'Animal Name'
    
    # Returns updated Map elements with new data
    return [
        dl.Map(style={'width': '750px', 'height': '500px'},
               # Centers the map on AAC coords
               center = [30.75, -97.48], 
               zoom=10,
               children=[dl.TileLayer(id="base-layer-id"),
                         # Marker with tool tip and popup based off
                         # of selected row's data
                         dl.Marker(position=[selectionLat, selectionLon],
                                   children=[dl.Tooltip(dff.loc[selected_row_index,'breed']),
                                             dl.Popup([html.H1(popupTag),
                                                       html.P(dff.loc[selected_row_index,'name'])
                                             ])
                                    ])   
                ])
    ]
if __name__ == '__main__':
    app.run_server()


 * Serving Flask app "Animal Rescue Dashboard - Justin Holmes" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Aug/2022 03:22:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2022 03:22:02] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2022 03:22:02] "GET /_dash-layout HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 2897, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 107, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 131, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1607, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1614, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'breed'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = s

127.0.0.1 - - [15/Aug/2022 03:22:03] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [15/Aug/2022 03:22:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2022 03:22:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2022 03:22:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2022 03:22:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2022 03:24:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2022 03:24:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2022 03:24:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2022 03:24:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2022 03:26:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2022 03:26:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2022 03:26:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.

       1 age_upon_outcome animal_id animal_type  \
0   5411         5 months   A669643         Dog   
1   3767          4 years   A712291         Dog   
2   1918          3 years   A698931         Dog   
3    622          2 years   A741441         Dog   
4   4723         8 months   A731795         Dog   
5   7840           1 year   A654132         Dog   
6   3964          2 years   A751717         Dog   
7   6557         6 months   A765461         Dog   
8   5032          2 years   A765283         Dog   
9   5433          2 years   A673611         Dog   
10  2275           1 year   A668767         Dog   
11  4404          4 years   A744935         Dog   
12  5273         5 months   A730258         Dog   
13  9544           1 year   A694389         Dog   
14  9655           1 year   A674720         Dog   
15   256          3 years   A717863         Dog   
16  1765          3 years   A757370         Dog   
17  6401         5 months   A758803         Dog   
18  9960         5 months   A68

127.0.0.1 - - [15/Aug/2022 03:26:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2022 03:26:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2022 03:26:51] "POST /_dash-update-component HTTP/1.1" 200 -


       1 age_upon_outcome animal_id animal_type  \
0   5411         5 months   A669643         Dog   
1   3767          4 years   A712291         Dog   
2   1918          3 years   A698931         Dog   
3    622          2 years   A741441         Dog   
4   4723         8 months   A731795         Dog   
5   7840           1 year   A654132         Dog   
6   3964          2 years   A751717         Dog   
7   6557         6 months   A765461         Dog   
8   5032          2 years   A765283         Dog   
9   5433          2 years   A673611         Dog   
10  2275           1 year   A668767         Dog   
11  4404          4 years   A744935         Dog   
12  5273         5 months   A730258         Dog   
13  9544           1 year   A694389         Dog   
14  9655           1 year   A674720         Dog   
15   256          3 years   A717863         Dog   
16  1765          3 years   A757370         Dog   
17  6401         5 months   A758803         Dog   
18  9960         5 months   A68

127.0.0.1 - - [15/Aug/2022 03:26:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2022 03:27:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2022 03:27:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2022 03:27:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2022 03:28:58] "POST /_dash-update-component HTTP/1.1" 200 -
